In [55]:
import pandas as pd
import sqlite3
import re
conn = sqlite3.connect("profiles.db")

def convert_class_rank(cr_str):
    cr = cr_str.split(' ')
    return int(cr[0])/int(cr[2])

def get_type_map(df_column):
    return {t:i for i, t in enumerate(set(df_column))}

df = pd.read_sql_query("SELECT * FROM profiles", conn)
df = df.dropna()
df = df[df.class_rank.str.match('\d* (of) \d*', na=False)]
df['class_rank'] = df['class_rank'].apply(lambda x: convert_class_rank(x))
replace_columns = ['hs_type', 'gender', 'status', 'hs_state', 'school']
for col in replace_columns:
    df[col].replace(get_type_map(df[col]), inplace=True)
print(df)

       school  year  gender  hs_type  hs_state  gpa_w  gpa_uw  sat_m  sat_r  \
3          45  2010       2        2         8   4.50    3.90  660.0  630.0   
19         45  2013       2        2         8   4.00    3.80  750.0  690.0   
22         45  2013       0        2         8   4.00    3.70  630.0  460.0   
47         45  2017       2        2         8   3.90    3.80  580.0  460.0   
59         45  2018       2        2         8   4.00    3.90  590.0  530.0   
73         45  2020       2        2         8   4.00    3.30  600.0  550.0   
121        40  2014       0        2        46   3.70    3.40  630.0  680.0   
133        40  2018       2        2        46   3.70    3.55  440.0  470.0   
134        40  2018       2        2         8   3.90    3.50  520.0  560.0   
136        40  2019       2        1        44   4.21    3.60  550.0  660.0   
143        71  2011       2        2         8   3.10    3.10  550.0  700.0   
147        71  2013       0        2         8   4.5